In [2]:
import pandas as pd
import re
import os ,sys

In [14]:
def read_xlsx(file_path:str, index = False)->pd.DataFrame:
    df = pd.read_excel(file_path, header=0,index_col = None )
    print(df.info())
    return df




def record_to_json(path:str):
    """
    Converts a given record file in .xlsx format to JSON format.

    Parameters:
    - path (str): The path to the record file.

    Returns:
    None
    """
    json_path = os.path.splitext(path)[0]+".json"
    data = read_xlsx(path).to_json(json_path,orient="records", indent=4) # here indent=4 makes it prettier
    print(f"Data saved to {json_path}")


In [4]:
def image_url_parser(recipe_card:str)->str:

    pattern = r'https[^"]+\.jpg'
    print(recipe_card)
    match = re.findall(pattern, recipe_card)
    if match:
        return match[0]

    return None

## FLOW
1- create an xlsx reformatter pipeline
  - unnecessary features
  - missing rows (any of the wanted col_names)
  - feature engineering via regex and dict indexing

2- record to json, orient record, indent = 4

In [15]:
df = read_xlsx("vegan_main-dishes.xlsx" )
df.head(10)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 61 entries, 0 to 60
Data columns (total 11 columns):
 #   Column                    Non-Null Count  Dtype 
---  ------                    --------------  ----- 
 0   web-scraper-order         61 non-null     object
 1   web-scraper-start-url     61 non-null     object
 2   sub_category              1 non-null      object
 3   card                      60 non-null     object
 4   card-href                 60 non-null     object
 5   recipe_name               60 non-null     object
 6   recipe_details_table      60 non-null     object
 7   recipe_ingredients        60 non-null     object
 8   recipe_directions         60 non-null     object
 9   recipe_tags               60 non-null     object
 10  recipe_nutrition_details  60 non-null     object
dtypes: object(11)
memory usage: 5.4+ KB
None


,web-scraper-order,web-scraper-start-url,sub_category,card,card-href,recipe_name,recipe_details_table,recipe_ingredients,recipe_directions,recipe_tags,recipe_nutrition_details
0,1699656347-1,https://www.allrecipes.com/recipes/1661/everyd...,Vegan Main Dishes,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,1699656350-2,https://www.allrecipes.com/recipes/1661/everyd...,NaN,"<img\nsrc=""https://www.allrecipes.com/thmb/kFt...",https://www.allrecipes.com/recipe/245424/mujad...,Mujadarra (Lentils with Rice),"[{""recipe_details_table"":""Prep Time:\n15 mins\...","[{""recipe_ingredients"":""14 ounces dry brown le...","[{""recipe_directions"":""Fill a large pot with l...","[{""recipe_tags"":""Recipes""},{""recipe_tags"":""Eve...","[{""recipe_nutrition_details"":""350\nCalories\n\..."
2,1699656354-3,https://www.allrecipes.com/recipes/1661/everyd...,NaN,"<img\nsrc=""https://www.allrecipes.com/thmb/W9m...",https://www.allrecipes.com/recipe/278267/beyon...,Beyond Beef Vegan Meatballs,"[{""recipe_details_table"":""Prep Time:\n25 mins\...","[{""recipe_ingredients"":""1 (10 ounce) package f...","[{""recipe_directions"":""Combine frozen spinach ...","[{""recipe_tags"":""Recipes""},{""recipe_tags"":""Mai...","[{""recipe_nutrition_details"":""411\nCalories\n\..."
3,1699656357-4,https://www.allrecipes.com/recipes/1661/everyd...,NaN,"<img\nsrc=""https://www.allrecipes.com/thmb/8hE...",https://www.allrecipes.com/recipe/263128/jicam...,Jicama Tortillas,"[{""recipe_details_table"":""Prep Time:\n15 mins\...","[{""recipe_ingredients"":""1 large jicama""}]","[{""recipe_directions"":""Wash jicama thoroughly ...","[{""recipe_tags"":""Recipes""},{""recipe_tags"":""Cui...","[{""recipe_nutrition_details"":""25\nCalories\n\n..."
4,1699656360-5,https://www.allrecipes.com/recipes/1661/everyd...,NaN,"<img\nsrc=""https://www.allrecipes.com/thmb/xvp...",https://www.allrecipes.com/recipe/232210/cousc...,Couscous with Olives and Sun-Dried Tomato,"[{""recipe_details_table"":""Prep Time:\n20 mins\...","[{""recipe_ingredients"":""1 ¼ cups vegetable bro...","[{""recipe_directions"":""Bring 1 1/4 cup vegetab...","[{""recipe_tags"":""Cuisine""},{""recipe_tags"":""Afr...","[{""recipe_nutrition_details"":""528\nCalories\n\..."
5,1699656363-6,https://www.allrecipes.com/recipes/1661/everyd...,NaN,"<img\nsrc=""https://www.allrecipes.com/thmb/0ZE...",https://www.allrecipes.com/recipe/25331/americ...,American-Style Red Beans and Rice,"[{""recipe_details_table"":""Prep Time:\n5 mins\n...","[{""recipe_ingredients"":""1 tablespoon olive oil...","[{""recipe_directions"":""Combine olive oil, kidn...","[{""recipe_tags"":""Recipes""},{""recipe_tags"":""Mai...","[{""recipe_nutrition_details"":""511\nCalories\n\..."
6,1699656366-7,https://www.allrecipes.com/recipes/1661/everyd...,NaN,"<img\nsrc=""https://www.allrecipes.com/thmb/xf0...",https://www.allrecipes.com/recipe/264316/veggi...,Veggie Stir-Fry with Potatoes,"[{""recipe_details_table"":""Prep Time:\n20 mins\...","[{""recipe_ingredients"":""2 pounds red potatoes,...","[{""recipe_directions"":""Place potatoes into a l...","[{""recipe_tags"":""Recipes""},{""recipe_tags"":""Cui...","[{""recipe_nutrition_details"":""260\nCalories\n\..."
7,1699656369-8,https://www.allrecipes.com/recipes/1661/everyd...,NaN,"<img\nsrc=""https://www.allrecipes.com/thmb/Lyx...",https://www.allrecipes.com/recipe/281154/quick...,Quick and Easy Pineapple Fried Rice,"[{""recipe_details_table"":""Prep Time:\n15 mins\...","[{""recipe_ingredients"":""2 tablespoons vegetabl...","[{""recipe_directions"":""Heat oil in a large ski...","[{""recipe_tags"":""Recipes""},{""recipe_tags"":""Mai...","[{""recipe_nutrition_details"":""239\nCalories\n\..."
8,1699656372-9,https://www.allrecipes.com/recipes/1661/everyd...,NaN,"<img\nsrc=""https://www.allrecipes.com/thmb/ghc...",https://www.allrecipes.com/recipe/278523/reduc...,Reduced-Calorie Chocolate Banana Oatmeal,"[{""recipe_details_table"":""Prep Time:\n5 mins\n...","[{""recipe_ingredients"":""2 tablespoons unsweete...","[{""recipe_directi

In [7]:
features = df.columns

In [8]:
features = ['sub_category', 'card',
       'card-href', 'recipe_name', 'recipe_details_table',
       'recipe_ingredients', 'recipe_directions', 'recipe_tags',
       'recipe_nutrition_details']

Index(['web-scraper-order', 'web-scraper-start-url', 'sub_category', 'card',
       'card-href', 'recipe_name', 'recipe_details_table',
       'recipe_ingredients', 'recipe_directions', 'recipe_tags',
       'recipe_nutrition_details'],
      dtype='object')

In [164]:
#sub_tag = df.loc[0, "sub_category"]
sub_tag
df["sub_category"] = sub_tag

In [ ]:
df["recipe_tags"][1:-1].map(lambda tags: eval(tags))

In [26]:
[kv["recipe_tags"]  for kv   in eval(df["recipe_tags"][1])]

['Recipes', 'Everyday Cooking', 'Vegan']

In [31]:
df.drop(df[df["recipe_tags"].isnull()].index, inplace=True)


In [34]:

df["recipe_tags_formatted"] = df["recipe_tags"].apply(lambda tags: [kv["recipe_tags"] for kv in eval(tags)])


In [66]:
df.drop(df[~df["recipe_details_table"].str.contains("Time")].index, inplace=True)

In [37]:
df["recipe_details_formatted"] = df["recipe_details_table"].apply(lambda tags: [kv["recipe_details_table"] for kv in eval(tags)])

In [71]:
df.drop(df[df["recipe_ingredients"].apply(lambda element: len(element))<=2].index, inplace=True)

In [74]:
df["recipe_ingredients_formatted"] = df["recipe_ingredients"].apply(lambda tags: [kv["recipe_ingredients"] for kv in eval(tags)])


In [91]:
from typing import List
def recipe_ingredients_formatter(ingredients:List)->List:
  """assumes the list has 1 element, if it contains multiple elements, then just ' '.join()  them """
  ingredient_str = ingredients[0]
  ingredients_list = ingredient_str.split('\n')

  # Remove empty strings and strip any leading/trailing whitespace
  ingredients_list = [ingredient.strip() for ingredient in ingredients_list if ingredient.strip()]
  return ingredients_list


In [92]:
recipe_ingredients_formatter(df["recipe_ingredients_formatted"][7])

['2 tablespoons vegetable oil',
 '½ cup diced sweet onion',
 '1 tablespoon grated ginger root',
 '1 clove garlic, minced',
 '1 ½ cups fresh pineapple chunks',
 '½  diced red bell pepper',
 '⅓ cup thawed frozen peas',
 '2 cups cold, cooked white rice',
 '2 tablespoons low-sodium soy sauce, or to taste',
 '1 teaspoon sesame oil',
 '2 tablespoons sliced green onion',
 '1 teaspoon toasted sesame seeds']

In [97]:
#formatting the whole column
df["recipe_ingredients_formatted"] = df["recipe_ingredients_formatted"].apply(lambda ingredient_list: recipe_ingredients_formatter(ingredient_list))


In [100]:
df.drop(df[df["recipe_directions"].apply(lambda element: len(element))<=2].index, inplace=True)

In [102]:
df["recipe_directions_formatted"] = df["recipe_directions"].apply(lambda tags: [kv["recipe_directions"] for kv in eval(tags)])


In [108]:
len(df["recipe_directions_formatted"][2]), df["recipe_directions_formatted"][2]

(6,
 ['Combine frozen spinach and water in a 1-quart microwave-safe casserole dish. Cover and microwave on high for 5 minutes, stirring halfway through the cooking time. Break up any clumps with a fork and cook for 1 more minute. Transfer to a colander to drain. When cool enough to handle, squeeze small handfuls of spinach to release remaining water and place spinach in a large mixing bowl.',
  'Add Beyond Beef, nutritional yeast, bread crumbs, tomato paste, minced garlic, oregano, basil, onion powder, salt, and pepper to the spinach. Mix with your hands until all ingredients are thoroughly combined. Roll into 1 1/4-inch diameter meatballs.',
  'Preheat oven to 400 degrees F (200 degrees C). Line a large rimmed baking sheet with parchment paper.',
  'Heat 1 tablespoon olive oil over medium heat in a large Dutch oven. Working in batches, add about 12 meatballs and cook, turning often and being careful not to burn, until browned on all sides, about 5 to 7 minutes. Transfer meatballs to t

In [110]:
df.drop(df[df["recipe_nutrition_details"].apply(lambda element: len(element))<=2].index, inplace=True)

In [112]:
df["recipe_nutrition_details_formatted"] = df["recipe_nutrition_details"].apply(lambda tags: [kv["recipe_nutrition_details"] for kv in eval(tags)])


In [135]:
df["recipe_nutrition_details_formatted"].iloc[0]

['350', 'Calories', '7g', 'Fat', '57g', 'Carbs', '16g', 'Protein']

In [153]:
def recipe_nutrition_formatter(nutrition_details:List)->List:
  nutrition_info = {
      "calories":0,
      "fat" : 0,
      "carbs":0,
      "protein":0}

  for index, item in enumerate(nutrition_details):
    if item.lower() in nutrition_info:
      nutrition_info[item.lower()] = nutrition_details[index-1]

  return  nutrition_info

In [154]:
recipe_nutrition_formatter(df["recipe_nutrition_details_formatted"].iloc[0])

{'calories': '350', 'fat': '7g', 'carbs': '57g', 'protein': '16g'}

In [ ]:
df["recipe_nutrition_details_formatted"] = df["recipe_nutrition_details_formatted"].apply( lambda element:  recipe_nutrition_formatter(element))



In [171]:
df.head(5).iloc[:,2:]

,sub_category,card,card-href,recipe_name,recipe_details_table,recipe_ingredients,recipe_directions,recipe_tags,recipe_nutrition_details,recipe_tags_formatted,recipe_details_formatted,recipe_ingredients_formatted,recipe_directions_formatted,recipe_nutrition_details_formatted
1,Vegan Main Dishes,"<img\nsrc=""https://www.allrecipes.com/thmb/kFt...",https://www.allrecipes.com/recipe/245424/mujad...,Mujadarra (Lentils with Rice),"[{""recipe_details_table"":""Prep Time:\n15 mins\...","[{""recipe_ingredients"":""14 ounces dry brown le...","[{""recipe_directions"":""Fill a large pot with l...","[{""recipe_tags"":""Recipes""},{""recipe_tags"":""Eve...","[{""recipe_nutrition_details"":""350\nCalories\n\...","[Recipes, Everyday Cooking, Vegan, Vegan Main ...",[Prep Time:\n15 mins\n\n\nCook Time:\n30 mins\...,"[14 ounces dry brown lentils, 2 carrots, grat...",[Fill a large pot with lightly salted water; a...,"{'calories': '350', 'fat': '7g', 'carbs': '57g..."
2,Vegan Main Dishes,"<img\nsrc=""https://www.allrecipes.com/thmb/W9m...",https://www.allrecipes.com/recipe/278267/beyon...,Beyond Beef Vegan Meatballs,"[{""recipe_details_table"":""Prep Time:\n25 mins\...","[{""recipe_ingredients"":""1 (10 ounce) package f...","[{""recipe_directions"":""Combine frozen spinach ...","[{""recipe_tags"":""Recipes""},{""recipe_tags"":""Mai...","[{""recipe_nutrition_details"":""411\nCalories\n\...","[Recipes, Main Dishes, Meatball Recipes, Vegan...",[Prep Time:\n25 mins\n\n\nCook Time:\n 1 hr\n\...,"[1 (10 ounce) package frozen chopped spinach, ...",[Combine frozen spinach and water in a 1-quart...,"{'calories': '411', 'fat': '26g', 'carbs': '19..."
3,Vegan Main Dishes,"<img\nsrc=""https://www.allrecipes.com/thmb/8hE...",https://www.allrecipes.com/recipe/263128/jicam...,Jicama Tortillas,"[{""recipe_details_table"":""Prep Time:\n15 mins\...","[{""recipe_ingredients"":""1 large jicama""}]","[{""recipe_directions"":""Wash jicama thoroughly ...","[{""recipe_tags"":""Recipes""},{""recipe_tags"":""Cui...","[{""recipe_nutrition_details"":""25\nCalories\n\n...","[Recipes, Cuisine, Latin American, Mexican, Ve...",[Prep Time:\n15 mins\n\n\nTotal Time:\n15 mins...,[1 large jicama],[Wash jicama thoroughly with cool water and re...,"{'calories': '25', 'fat': '0g', 'carbs': '6g',..."
4,Vegan Main Dishes,"<img\nsrc=""https://www.allrecipes.com/thmb/xvp...",https://www.allrecipes.com/recipe/232210/cousc...,Couscous with Olives and Sun-Dried Tomato,"[{""recipe_details_table"":""Prep Time:\n20 mins\...","[{""recipe_ingredients"":""1 ¼ cups vegetable bro...","[{""recipe_directions"":""Bring 1 1/4 cup vegetab...","[{""recipe_tags"":""Cuisine""},{""recipe_tags"":""Afr...","[{""recipe_nutrition_details"":""528\nCalories\n\...","[Cuisine, African, North African, Egyptian, Ve...",[Prep Time:\n20 mins\n\n\nCook Time:\n30 mins\...,"[1 ¼ cups vegetable broth, 1 ¼ cups water, 2 c...",[Bring 1 1/4 cup vegetable broth and water to ...,"{'calories': '528', 'fat': '29g', 'carbs': '56..."
5,Vegan Main Dishes,"<img\nsrc=""https://www.allrecipes.com/thmb/0ZE...",https://www.allrecipes.com/recipe/25331/americ...,American-Style Red Beans and Rice,"[{""recipe_details_table"":""Prep Time:\n5 mins\n...","[{""recipe_ingredients"":""1 tablespoon olive oil...","[{""recipe_directions"":""Combine olive oil, kidn...","[{""recipe_tags"":""Recipes""},{""recipe_tags"":""Mai...","[{""recipe_nutrition_details"":""511\nCalories\n\...","[Recipes, Main Dishes, Rice, Beans and Rice Re...",[Prep Time:\n5 mins\n\n\nCook Time:\n30 mins\n...,"[1 tablespoon olive oil, 1 (15 ounce) can kidn...","[Combine olive oil, kidney beans, tomato sauce...","{'calories': '511', 'fat': '5g', 'carbs': '101..."


In [ ]:
df["recipe_tags_formatted"].iloc[1:].apply(lambda element: element.append("Vegan Main Dishes"))